In [1]:
from pathlib import Path
import polars as pl
import duckdb
from datasets import Dataset
import numpy as np
from autofaiss import build_index
from FlagEmbedding import BGEM3FlagModel
from core_pro.ultilities import make_dir, make_sync_folder

In [2]:
path = make_sync_folder('Item_Matching_Test')
file = path / 'clean.parquet'

query = f"""select * from read_parquet('{file}')"""
df = duckdb.sql(query).pl()
df.head()

item_id,item_name,shop_id,shop_name,level1_global_be_category,description,images,image_url,item_name_clean,file_path,exists
i64,str,i64,str,str,str,str,str,str,str,bool
22333354558,"""chân váy jean bò nữ chữ a jean…",691586816,"""Jeanshop_0603""","""Women Clothes""","""1. Thông tin sản phẩm: Chân vá…","""vn-11134207-7r98o-lsnlwcvto3ic…","""http://f.shopee.vn/file/vn-111…","""chân váy jean bò nữ chữ a jean…","""/Users/kevinkhang/Downloads/Da…",true
19683965502,"""Quần ống rộng vải tăm lạnh mềm…",24751730,"""Lylyshop Unisex Cơ sở 1""","""Women Clothes""","""[{""t"":""QUẦN FORM UNISEX DÀNH C…","""vn-11134207-7r98o-lmwyou2s1qsf…","""http://f.shopee.vn/file/vn-111…","""quần ống rộng vải tăm lạnh mềm…","""/Users/kevinkhang/Downloads/Da…",true
29208290885,"""Đầm 2s maxi phối ren cúp ngực …",158092907,"""Loulou Clothing""","""Women Clothes""","""[{""t"":""Chất larong phối ren. \…","""vn-11134207-7r98o-lz4yagwociw1…","""http://f.shopee.vn/file/vn-111…","""đầm 2s maxi phối ren cúp ngực …","""/Users/kevinkhang/Downloads/Da…",true
25329730279,"""Set Bộ Đồ Nữ Thun Gân Tăm Lạnh…",390933323,"""HÀ-THU SHOP""","""Women Clothes""","""[{""t"":""Set bộ ống rộng thun gâ…","""vn-11134207-7r98o-lv4h7q6slbx6…","""http://f.shopee.vn/file/vn-111…","""set bộ đồ nữ thun gân tăm lạnh…","""/Users/kevinkhang/Downloads/Da…",true
22031178913,"""Quần jean nữ ống suông rộng Cạ…",965335291,"""Daisy-Clothing""","""Women Clothes""","""Quần jean nữ ống suông rộng Cạ…","""vn-11134207-7r98o-lsqfyrydb1cp…","""http://f.shopee.vn/file/vn-111…","""quần jean nữ ống suông rộng cạ…","""/Users/kevinkhang/Downloads/Da…",true


In [3]:
name = 'bge'
path_tmp_array = Path(path / f'tmp/array/{name}')
path_tmp_ds = Path(path / f'tmp/ds/{name}')
make_dir(path_tmp_ds)
make_dir(path_tmp_array)

file_embed = path_tmp_array / 'embed.npy'
if not file_embed.exists():
    model = BGEM3FlagModel('BAAI/bge-m3', use_fp16=False)
    embeddings = model.encode(
        df['item_name_clean'].to_list(),
        batch_size=8,
        max_length=80,
        return_dense=True,
        return_sparse=False,
        return_colbert_vecs=False
    )['dense_vecs']
    np.save(file_embed, embeddings)
else:
    embeddings = np.load(file_embed)
print(embeddings.shape)

(9936, 1024)


In [4]:
df = df.with_columns(pl.Series(values=embeddings, name='embed'))
dataset = Dataset.from_polars(df)
dataset.set_format(type='numpy', columns=['embed'], output_all_columns=True)

In [5]:
path_index = Path(path / 'tmp/index')
build_index(
    embeddings=embeddings,
    index_path=str(path_index / f'ip.index'),
    index_infos_path=str(path_index / f'index.json'),
    save_on_disk=True,
    metric_type='ip',
    verbose=30,
)

  0%|          | 0/1 [00:00<?, ?it/s]


(<faiss.swigfaiss_avx2.IndexHNSWFlat; proxy of <Swig Object of type 'faiss::IndexHNSWFlat *' at 0x7b8858053ea0> >,
 {'index_key': 'HNSW15',
  'index_param': 'efSearch=5226',
  'index_path': '/home/kevin/Downloads/Item_Matching_Test/tmp/index/ip.index',
  'size in bytes': 42050218,
  'avg_search_speed_ms': 9.97823628438878,
  '99p_search_speed_ms': 10.557658101897687,
  'reconstruction error %': 0.0,
  'nb vectors': 9936,
  'vectors dimension': 1024,
  'compression ratio': 0.9678393581693203})

In [6]:
# add index
dataset.load_faiss_index('embed', path_index / f'ip.index')

In [7]:
score, result = dataset.get_nearest_examples_batch(
    'embed',
    np.asarray(dataset['embed']),
    k=5
)

dict_ = {'score': [list(i) for i in score]}
df_score = pl.DataFrame(dict_)
df_result = (
    pl.DataFrame(result).drop(['embed'])
    .select(pl.all().name.prefix(f'db_'))
)

In [8]:
df_match = pl.concat([df, df_result, df_score], how='horizontal')
col_explode = [i for i in df_match.columns if 'notebooks' in i] + ['score']

In [10]:
path_export = path / 'text_match'
make_dir(path_export)
df_match.write_parquet(path_export / f'{name}.parquet')